In [1]:
import numpy as np

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20
from bokeh.models import PrintfTickFormatter

from ipywidgets import FloatProgress, IntProgress
from IPython.display import display

from mylib.thermal_explosion_model import thermal_explosion_model, fuel_thermal_explosion_model

import mylib.integration as integration

output_notebook(hide_banner=True)

# Themal explosion 

## Model with fuel consumption neglected

\begin{equation}
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta = \exp(\theta)
\end{equation}

### Integration with Radau5

In [17]:
def thermal_explosion_radau():

    lamb = 0.88
    xmin = 0.
    xmax = 2.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 40

    tem = thermal_explosion_model(lamb=lamb, xmin=xmin, xmax=xmax, nx=nx)
    fcn_radau = tem.fcn_radau

    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = np.zeros(nx)
    fig_sol = figure(x_range=(xmin, xmax), plot_height=500, plot_width=950, 
                     title="Solution (click on legend to hide correponding graph)")
    fig_sol.x(x, yini, color="Grey", legend_label="sol at t = 0")
    fig_sol.legend.click_policy="hide"
    
    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    for it in range(nt_plot-1):
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn_radau, njac=1, atol=1.e-6, rtol=1.e-6)
        yini = sol.y
        fig_sol.x(x, sol.y, legend_label=f"sol at t={t_plot[it+1]:.2f}", color=Category10[10][it])
        
    show(fig_sol, notebook_handle=True)
     
thermal_explosion_radau()

### Integration with Rock4

In [3]:
def thermal_explosion_rock():

    lamb = 0.86
    xmin = 0.
    xmax = 2.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 1.0

    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)

    tem = thermal_explosion_model(lamb=lamb, xmin=xmin, xmax=xmax, nx=nx)
    fcn_rock = tem.fcn_rock
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = np.zeros(nx)
    fig_sol = figure(x_range=(xmin, xmax), plot_height=500, plot_width=950, 
                     title="Solution (click on legend to hide correponding graph)")
    fig_sol.x(x, yini, color="Grey", legend="sol at t = 0")
    fig_sol.legend.click_policy="hide"

    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    for it in range(nt_plot-1):
        sol = integration.rock4(t_plot[it], t_plot[it+1], yini, fcn_rock, tol=1.e-6)
        yini = sol.y
        fig_sol.x(x, sol.y, legend=f"sol at t={t_plot[it+1]:.2f}", color=Category10[10][it])
        progress_bar.value = t_plot[it+1]
        
    show(fig_sol, notebook_handle=True)
     
thermal_explosion_rock()

FloatProgress(value=0.0, description='Progress:', max=1.0)

Exception ignored on calling ctypes callback function: <bound method thermal_explosion_model.fcn_rock of <mylib.thermal_explosion_model.thermal_explosion_model object at 0x7f6287957f40>>
Traceback (most recent call last):
  File "/home/paul/Documents/cours/3A/MAP551/MAP551/PC7/map551_notebook_pc07-master/mylib/thermal_explosion_model.py", line 52, in fcn_rock
    ydot[ix] = oneoverlambdxdx*(y[ix-1] - 2*y[ix] + y[ix+1]) + np.exp(y[ix])
KeyboardInterrupt: 


## Model with fuel consumption

$$
\left\{\begin{aligned}
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta & = \exp(\theta)\, Y\\
\partial_\tau Y - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} Y & = - \varepsilon \exp(\theta)\, Y
\end{aligned}\right.
$$

In [26]:
def fuel_thermal_explosion_radau():

    lamb = 0.9
    eps = 0.002
    xmin = 0.
    xmax = 2.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 3.0
    neq = 2

    ftem = fuel_thermal_explosion_model(lamb=lamb, eps=eps, xmin=xmin, xmax=xmax, nx=nx)
    fcn_radau = ftem.fcn_radau

    yini = np.zeros((nx, neq))
    for inx in range(nx):
        yini[inx, 1] = 0.5+inx/(nx)
        #yini[inx, 1] = 1.0
    yini = yini.flatten()
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin, xmax, nx)
    fig_sol_01 = figure(x_range=(xmin, xmax), plot_height=450, plot_width=950, 
                         title="theta (click on legend to hide correponding graph)")
    fig_sol_02 = figure(x_range=(xmin, xmax), plot_height=450, plot_width=950, 
                         title="Y (click on legend to hide correponding graph)")
    fig_sol_01.x(x, yini.reshape(nx,neq)[:,0], color="Grey", legend_label="sol at t = 0")
    fig_sol_02.x(x, yini.reshape(nx,neq)[:,1], color="Grey", legend_label="sol at t = 0")
    fig_sol_01.legend.click_policy="hide"
    fig_sol_02.legend.click_policy="hide"
    fig_sol_02.legend.location = "top_left"

    # radau integration
    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    for it in range(nt_plot-1):
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn_radau, njac=2, atol=1.e-6, rtol=1.e-6)
        yini = sol.y
        theta = sol.y.reshape(nx,neq)[:,0]
        Y = sol.y.reshape(nx,neq)[:,1]
        fig_sol_01.x(x, theta, legend_label=f"sol at t={t_plot[it+1]:.2f}", color=Category10[10][it])
        fig_sol_02.x(x, Y, legend_label=f"sol at t={t_plot[it+1]:.2f}", color=Category10[10][it])
        
    show(column(fig_sol_01, fig_sol_02))

fuel_thermal_explosion_radau()